# Funktionen: Gültigkeitsbereich von Variablen (Scope)

Bisher haben wir uns noch keine Gedanken darüber gemacht, wann und wo der Wert einer Variable sichtbar ist. In Zusammenhang mit Funktionen müssen wir uns jedoch damit beschäftigen. Vorauszuschicken ist, dass diese Sichtbarkeit in Python eher ungewöhnlich gelöst ist.

Grundsätzlich müssen wir zwei Gültigkeitsbereiche unterscheiden:

  1) den globalen Scope: globale Werte sind an allen Stellen des Programms sichtbar
     (im Sinne von "können überall gelesen werden")
  3) den lokalen Scope: Werte im lokalen Scope können nur in Teilen des Programms verwendet
     werden: im Normalfall innerhalb der Funktion, in der sie angelegt wurden.

## Der globale Gültigkeitsbereich

Wie der Name schon vermuten lässt, sind **globale Variablen** überall (außerhalb und innerhalb von Funktionen) sichtbar. Sie werden üblicherweise außerhalb einer Funktion angelegt:

In [ ]:
VAT = 20

def calculate_brutto(price:float) -> float:
    "Return price including VAT."
    return price + price * VAT / 100

print(calculate_brutto(7.50))

In diesem Beispiel haben wir der Funktion ``calculate_brutto()`` den Netto-Preis übergeben und den finalen Preis (inklusive Umsatzsteuer) zurückbekommen. Die Höhe der Umsatzsteuer (``VAT``: 20%) wurde im Unterschied zu `price` nicht an die Funktion übergeben, sondern außerhalb der Funktion als globale Variable angelegt. Eine solche globale Variable ist auch innerhalb der Funktion sichtbar, weshalb in der Funktion mit diesem Wert gerechnet werden kann.

*Kleiner Exkurs:* Viele Programmiersprachen kennen **Konstanten**. Das sind Werte, die nur einmal festgelegt und später nicht mehr verändert werden können. Python kennt dieses Konzept nicht, sondern nur Variablen. Allerdings gibt es in Python die Konvention, dass Variablen, die mit Großbuchstaben geschrieben sind, wie Konstanten behandelt werden: Man sollte also den Wert einer in Großbuchstaben geschriebenen Variable **nie** im Programmablauf verändern.

### Vorsicht bei globalen Variablen

In der Praxis sollte man globale Variablen möglichst sparsam einsetzen. Die Gründe dafür sind:

  * Verwendet man viele globale Variablen, erschweren das die Nachvollziehbarkeit des Codes
  * Globale Variablen bergen die Gefahr von *Seiteneffekten*. Damit ist gemeint, dass man den Wert einer globalen Variable unabsichtlich verändert und
    damit einen schwer zu findenden Fehler produziert.

Die klaren Empfehlung ist daher, keine oder zumindest nur wenige globale Variablen zu verwenden und eher auf lokale Variablen mit eingeschränkten Gültigkeitsbereich zu setzen.

### Juyter Notebooks und globale Variablen

Ein unschöner Seiteneffekt von Jupyter Notebooks ist, dass hier oft unbewußt viele globale Variablen angelegt werden, weil Codezellen den Eindruck einer Strukturierung und Abgrenzung der Codeteile vermitteln. Diese Struktur hat aber keine Auswirkungen auf die Sichbarkeit von Variablen: Eine in einer Codezelle global (d.h. außerhalb einer Funktion) angelegte Variable ist, sobald die Zelle ausgeführt wurde, in allen anderen Zellen sichtbar und veränderbar. 

Das erhöht die Gefahr von unbeabsichtigen Seiteneffekten enorm und kann sogar dazu führen, dass die Ergebnisse eines Notebooks davon abhängen, in welcher Reihenfolge die Codezellen ausgeführt wurden. 

## Der lokale Gültigkeitsbereich

Beginnen wir mit einem kleinen Beispiel:

In [ ]:
def do_something():
    some_value = 42

do_something()
print(some_value)

Hier haben wir innerhalb der Funktion eine lokale Variable `some_value` angelegt. Wenn wir versuchen, außerhalb der Funktion auf diese Variable zuzugreifen, wirft Python einen ``NameError: name 'some_value' is not defined``. Mit anderen Worten: Die lokale Variable ``some_value`` ist im globalen Scope nicht sichtbar.

Da diese Gültigkeitsbereiche jeweils eigene Namensräume bilden, können wir im lokalen und globalen Scope unterschiedliche Variablen mit gleichem Namen haben. Es können also im lokalen und globalen Raum zwei Variablen mit unterschiedlichen Werten existieren, die denselben Name tragen. Die Verwendung gleicher Namen in unterschiedlichen Gültigkeitsbereichen wird jedoch aus Gründen, auf die ich später noch eingehen werde, nicht empfohlen.

Im folgenden Beispiel setzen wir im globalen Scope den Wert von `some_value` auf `1`. Innerhalb der Funktion legen wir eine zweite Variable `some_value` mit dem Wert `42` an.
Anhand der Ausgabe sehen wir, dass es sich hier wirklich um zwei unterschiedliche Variablen handelt, auch wenn sie denselben Namen tragen.

In [ ]:
def do_something():
    some_value = 42
    print(f"some_value im lokalen Scope: {some_value}")
    
some_value = 1
do_something()
print(f"some_value im globalen Scope: {some_value}")

### Parameterwerte

Was passiert nun, wenn wir einen Wert als Funktionsparameter übergeben? Schreiben wir dazu eine einfache Funktion ``double()``, die den übergebenen Wert verdoppelt:

In [ ]:
def double(val):
    print(f"val in der Funktion [b]: {val}")
    val += val
    print(f"val in der Funktion [c]: {val}")

val = "Wuff"
print(f"val außerhalb der Funktion [a]: {val}")
double(val)
print(f"val außerhalb der Funktion [d]: {val}")

Wir haben hier wieder denselben Variablennamen (`val`) in beiden  Gültigkeitsbereichen mit unterschiedlichen Werten. Der vedoppelte Wert wird nur bei [c] in der Funktion ausgegeben. Interessanter wird die Sache, wenn wir uns zusätzlich die ID der Werte ausgeben lassen, auf die das jeweilige ``val`` zeigt:

In [ ]:
def double(val):
    print(f"val in der Funktion [b]: {val}; ID: {id(val)}")
    val += val
    print(f"val in der Funktion [c]: {val}; ID: {id(val)}")

val = "Wuff"
print(f"val außerhalb der Funktion [a]: {val}; ID: {id(val)}")
double(val)
print(f"val außerhalb der Funktion [d]: {val}; ID: {id(val)}")

Hier sehen wir, dass das lokale ``val`` in der Funktion zunächst auf denselben Wert zeigt, wie das globale ``val``: Die beiden IDs ([a] und [b]) sind identisch. 

Das bedeutet, dass Python nicht eine Kopie des Wertes der globalen Variable an die Funktion übergeben hat, sondern eine Referenz auf den Wert. In Zeile 2 zeigt die lokale Variable daher auf denselben Wert wie die globale Variable. 

Wenn wir in Zeile 3 den Wert von ``val`` ändern, ändert sich auch die ID des Wertes ([c]). Das ist nicht weiter verwunderlich, weil Strings (so wie int, float usw.) zu den unveränderbaren Datentypen (immutable types) gehören: Wenn wir den Wert einer solchen Variable ändern, wird für den Wert ein neues Objekt (mit einer anderen ID) erzeugt.

Das verhindert, dass wir in einer Funktion irrtümlich den Wert einer globalen Variable ändern und so einen unbeabsichtigten Seiteneffekt erzeugen.

### Überlagerte Variablen

Im letzten Beispiel hat die lokale Variable (sprich: der Parameter der Funktion) denselben Namen wie die globale Variable. Das funktioniert technisch zwar, wird aber nicht empfohlen, weil die lokale Variable innerhalb der Funktion die globale Variable *überlagert*. Das erschwert die Nachvollziehbarkeit des Codes und macht die globale Variable innerhab der Funktion unsichtbar. Eine bessere Lösung wäre, dem Parameter einen anderen Namen zu geben:

In [ ]:
def double(val_to_double):
    print(f"val_to_double in der Funktion [b]: {val_to_double}; ID: {id(val_to_double)}")
    val_to_double += val_to_double
    print(f"val_to_double in der Funktion [c]: {val_to_double}; ID: {id(val_to_double)}")

val = "Wuff"
print(f"val außerhalb der Funktion [a]: {val}; ID: {id(val)}")
double(val)
print(f"val außerhalb der Funktion [d]: {val}; ID: {id(val)}")

Bei Bedarf können wir nun in der Funktion auf die globale Variable zugreifen, weil sie nicht mehr überlagert wird:

In [ ]:
def repeat(val_to_double):
    print(f"val_to_double in der Funktion [b]: {val_to_repeat}; ID: {id(val_to_repeat)}")
    val_to_double += val_to_double
    print(f"val_to_double in der Funktion [c]: {val_to_double}; ID: {id(val_to_double)}")
    print(f"val in der Funktion [e]: {val}; ID: {id(val)}")

val = "Wuff"
print(f"val außerhalb der Funktion [a]: {val}; ID: {id(val)}")
double(val)
print(f"val außerhalb der Funktion [d]: {val}; ID: {id(val)}")

### Parameterwerte und veränderbare Datentypen

Hat der an eine Funktion übergebene Wert einen veränderbaren Datentyp (mutable type), so können wir uns nicht drauf verlassen, dass Seiteneffekte verhindert werden. Wir haben bereits einige solche Datentypen kennengelernt: Listen (list), Dictionaries (dict) oder normale Sets (im Unterschied zu FrozenSets). Dazu gehören auch die meisten Objekte von selbst geschriebenen Klassen (kommt noch später). Was passiert aber nun, wenn wir so einen Datentyp an die Funktion übergeben?

In [ ]:
def double(val):
    print(f"val in der Funktion [b]: {val}; ID: {id(val)}")
    val += val
    print(f"val in der Funktion [c]: {val}; ID: {id(val)}")

val = ["Foo", "Bar"]
print(f"val außerhalb der Funktion [a]: {val}; ID: {id(val)}")
double(val)
print(f"val außerhalb der Funktion [d]: {val}; ID: {id(val)}")

Hier fällt auf, dass die IDs in allen Ausgabezeichen gleich sind. Mit anderen Worten: Wir haben es geschafft, in der Funktion (also vermeintlich im lokalen Scope) den globalen Wert zu verändern! Das ist, wie bereits beschrieben, relativ gefährlich, weil wir so Seiteneffekte und damit schwer zu findende Fehler produzieren können. 

Wie gehen wir nun mit solchen Fällen am besten um? Wir sollten

   * keine veränderbaren Typen (Listen, Dictionaries, ...) als Funktionsargumente verwendet
   * oder zumindest darauf achtet, dass diese innerhalb der 
     Funktion nicht verändert werden 
   * Alternativ kann man mit Kopien (siehe dazu das ``copy`` Modul der Standardlibrary)
     oder mit alternativen unveränderbaren Typen (wie Tupel) arbeiten.

Da ich schon mehrfach auf ein Missverständnis gestoßen bin: Die Verwendung eines anderen Parameternamens hilft hier nicht:

In [ ]:
def double(val_to_double):
    print(f"val_to_double in der Funktion [b]: {val_to_double}; ID: {id(val_to_double)}")
    val_to_double += val_to_double
    print(f"val_to_double in der Funktion [c]: {val_to_double}; ID: {id(val_to_double)}")
    

val = ["Foo", "Bar"]
print(f"val außerhalb der Funktion [a]: {val}; ID: {id(val)}")
double(val)
print(f"val außerhalb der Funktion [d]: {val}; ID: {id(val)}")

Wie wir sehen, haben wir jetzt zwar im lokalen Scope einen anderen Variablennamen, der referenzierte Wert ist aber immer noch derselbe wie im globalen Scope.

### Veränderbare Datentype als default Parameterwerte

Noch gefährliche als die Übergabe von veränderbaren Datentypen an eine Funktion ist, wenn wir einen veränderbaren Datentyp als Defaultwert für einen Parameter verwenden:

In [ ]:
def do_something(val, mylist = []):
    mylist.append(val)
    return mylist
    
val1 = do_something('foo')
print(f"val1: {val1}")

val2 = do_something('bar')
print(f"val2: {val2}")

# This might be a little surprising:
print(f"val1: {val1}")
    

In diesem Beispiel wird beim ersten Aufruf der Funktion unbeabsichtigt ein Wert im globalen Scope angelegt, der bei jedem Aufruf der Funktion weiterverwendet wird. Jeder Aufruf der Funktion erweitert also die Liste um ein Element. Da es sich um einen Wert im globalen Scope handelt, werden auch bereits früher befüllte Variablen (``val1``) verändert. Damit hat man einen Fehler produziert der richtig schwer zu finden ist.

Statt dessen sollte man diese Funktion so schreiben:

In [ ]:
def do_something(val, mylist = None):
    if mylist is None:
        mylist = []
    mylist.append(val)
    return mylist

val1 = do_something('foo')
print(f"val1: {val1}")

val2 = do_something('bar')
print(f"val2: {val2}")

# val1 is still on the original value
print(f"val1: {val1}")

Der Unterschied ist, dass ``mylist`` hier garantiert als lokale Variable angelegt wird, wenn nicht explizit ein Wert übergeben wurde.

## Vertiefende Literatur
Ich empfehle ausdrücklich, mindestens eine der folgenden Ressourcen zur Vertiefung zu lesen!

  * Python Tutorial: 
	* Kapitel 4.6 - Defining Functions 
      (https://docs.python.org/3/tutorial/controlflow.html#defining-functions)
    * Kapitel 4.7 - More on Defining Functions
	  (https://docs.python.org/3/tutorial/controlflow.html#more-on-defining-functions)
  * Klein, Kurs: 
	* Funktionen (https://python-kurs.eu/python3_funktionen.php)
	* Parameter-Übergabe (http://python-kurs.eu/python3_parameter.php)
	* Globale und lokale Variablen (http://python-kurs.eu/python3_global_lokal.php)
	* Rekursive Funktionen (http://python-kurs.eu/python3_rekursive_funktionen.php)
	* Flaches und tiefes Kopieren (http://python-kurs.eu/python3_deep_copy.php)
  * Sweigart: https://automatetheboringstuff.com/2e/chapter3/  
    
    
  * Klein, Buch: Kapitel 14, 15 und evtl. 13.
  * Kofler: Kapitel 9.
  * Inden: Kapitel 2.5.
  * Weigend: Kapitel 6.1 bis 6.8 und 6.14.
  * Pilgrim: Kapitel 1.2
    (https://www.diveinto.org/python3/your-first-python-program.html#declaringfunctions)
  * Downey: Kapitel 3
    (http://www.greenteapress.com/thinkpython/html/thinkpython004.html)
    
    
  * Video: Ned Batchelder - Facts and Myths about Python names and values - PyCon 
    2015 (https://www.youtube.com/watch?v=_AEJHKGk9ns)

## Lizenz

This notebook ist part of the course [Grundlagen der Programmierung](https://github.com/gvasold/gdp) held by [Gunter Vasold](https://online.uni-graz.at/kfu_online/wbForschungsportal.cbShowPortal?pPersonNr=51488) at Graz University 2017&thinsp;ff. 

<p>
    It is licensed under <a href="https://creativecommons.org/licenses/by-nc-sa/4.0">CC BY-NC-SA 4.0</a>
</p>

<table>
    <tr>
    <td>
        <img style="height:22px" 
             src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"/></li>
    </td>
    <td>
    <img style="height:22px;"
         src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1" /></li>
    </td>
    <td>
        <img style="height:22px;"
         src="https://mirrors.creativecommons.org/presskit/icons/nc.svg?ref=chooser-v1" /></li>
    </td>
    <td>
        <img style="height:22px;"
             src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1" /></li>
    </td>
</tr>
</table>

---

<div style="text-align: center; margin-top:20pt;">  
  <a href="01-funktionen_basics.ipynb">← Vorheriges Notebook</a>  
  <span style="width: 100px;display: inline-block;"> </span>
    <a href="00-inhalt.ipynb">↑ Inhaltsübersicht</a>
    <span style="width: 100px;display: inline-block;"> </span>
  <a href="../04-grundlagen-plus/01-ausnahmen.ipynb">Nächstes Notebook →</a>  
</div> 